# Flujo de pacientes en clínica
## Modelo DES
Parte práctica 1

### Configuración Inicial del Entorno

In [12]:
#!pip install simpy

In [13]:
import simpy
import random
import statistics

### Definición de Parámetros Globales

In [14]:
NUM_RECEPCIONISTAS = 1
NUM_MEDICOS = 2
TIEMPO_REGISTRO_PROMEDIO = 2.0  # minutos
TIEMPO_CONSULTA_PROMEDIO = 7.0  # minutos
TASA_LLEGADA_PACIENTES = 5.0  # un paciente cada 5.0 minutos en promedio
TIEMPO_SIMULACION = 120  # minutos

# Lista global para recolectar tiempos totales de cada paciente
tiempos_de_espera_totales = []

### Función de paciente
Actua como un generador de Python y define el proceso completo de un paciente a través de la clínica

In [15]:
def paciente(env, nombre, recepcionistas, medicos):
    """
    Proceso que representa el flujo de un paciente a través de la clínica.
    
    Args:
        env: Entorno de SimPy
        nombre: Identificador del paciente
        recepcionistas: Recurso de recepcionistas
        medicos: Recurso de médicos
    """
    tiempo_de_llegada = env.now # Registrar tiempo de llegada
    print(f'[{env.now:.2f}] {nombre} llega a la clínica')
    
    # Etapa de Registro
    print(f'[{env.now:.2f}] {nombre} solicita recepcionista')
    with recepcionistas.request() as request:
        # Esperar hasta que el recurso esté disponible
        yield request
        
        print(f'[{env.now:.2f}] {nombre} inicia registro')
        # Simular tiempo de registro con distribución exponencial
        tiempo_registro = random.expovariate(1.0 / TIEMPO_REGISTRO_PROMEDIO)
        yield env.timeout(tiempo_registro)
        
        print(f'[{env.now:.2f}] {nombre} termina registro (duró {tiempo_registro:.2f} min)')
    
    # Etapa de Consulta
    print(f'[{env.now:.2f}] {nombre} solicita médico')
    with medicos.request() as request:
        # Esperar hasta que un médico esté disponible
        yield request
        
        print(f'[{env.now:.2f}] {nombre} inicia consulta médica')
        # Simular tiempo de consulta con distribución exponencial
        tiempo_consulta = random.expovariate(1.0 / TIEMPO_CONSULTA_PROMEDIO)
        yield env.timeout(tiempo_consulta)
        
        print(f'[{env.now:.2f}] {nombre} termina consulta (duró {tiempo_consulta:.2f} min)')
    
    # Calcular y almacenar resultados
    tiempo_total_en_clinica = env.now - tiempo_de_llegada
    tiempos_de_espera_totales.append(tiempo_total_en_clinica)
    
    print(f'[{env.now:.2f}] {nombre} sale de la clínica (tiempo total: {tiempo_total_en_clinica:.2f} min)')


### Generador de pacientes

In [16]:
def generador_pacientes(env, recepcionistas, medicos):
    """
    Generador que crea nuevos pacientes a intervalos de tiempo aleatorios.
    
    Args:
        env: Entorno de SimPy
        recepcionistas: Recurso de recepcionistas
        medicos: Recurso de médicos
    """
    contador_pacientes = 0
    
    # Generador con bucle infinito
    while True:
        # Esperar tiempo aleatorio hasta la llegada del siguiente paciente
        # Distribución exponencial para llegadas tipo Poisson
        tiempo_entre_llegadas = random.expovariate(1.0 / TASA_LLEGADA_PACIENTES)
        yield env.timeout(tiempo_entre_llegadas)
        
        # Crear nuevo paciente
        contador_pacientes += 1
        nombre_paciente = f'Paciente_{contador_pacientes}'
        
        # Iniciar proceso del paciente
        env.process(paciente(env, nombre_paciente, recepcionistas, medicos))

### Ejecución principal

In [17]:
def ejecutar_simulacion():
    """
    Función principal que configura y ejecuta la simulación completa.
    """
    print("SIMULACIÓN DE CLÍNICA - INICIO")
    print(f"Parámetros: {NUM_RECEPCIONISTAS} recepcionista(s), {NUM_MEDICOS} médico(s)")
    print(f"Tiempo de simulación: {TIEMPO_SIMULACION} minutos")
    print()
    
    # Limpiar lista de tiempos (por si se ejecuta múltiples veces)
    global tiempos_de_espera_totales
    tiempos_de_espera_totales = []
    
    # Instanciar el entorno de SimPy
    env = simpy.Environment()
    
    # Crear recursos con capacidad especificada
    recepcionistas = simpy.Resource(env, capacity=NUM_RECEPCIONISTAS)
    medicos = simpy.Resource(env, capacity=NUM_MEDICOS)
    
    # Iniciar el proceso generador de pacientes
    env.process(generador_pacientes(env, recepcionistas, medicos))
    
    # Ejecutar la simulación
    env.run(until=TIEMPO_SIMULACION)
    
    return tiempos_de_espera_totales

In [18]:
# Establecer semilla para reproducibilidad (opcional, comentar para aleatorio)
random.seed(42)

resultados = ejecutar_simulacion()

SIMULACIÓN DE CLÍNICA - INICIO
Parámetros: 1 recepcionista(s), 2 médico(s)
Tiempo de simulación: 120 minutos

[5.10] Paciente_1 llega a la clínica
[5.10] Paciente_1 solicita recepcionista
[5.10] Paciente_1 inicia registro
[5.23] Paciente_2 llega a la clínica
[5.23] Paciente_2 solicita recepcionista
[5.74] Paciente_1 termina registro (duró 0.64 min)
[5.74] Paciente_1 solicita médico
[5.74] Paciente_1 inicia consulta médica
[5.74] Paciente_2 inicia registro
[6.49] Paciente_3 llega a la clínica
[6.49] Paciente_3 solicita recepcionista
[8.00] Paciente_2 termina registro (duró 2.26 min)
[8.00] Paciente_2 solicita médico
[8.00] Paciente_2 inicia consulta médica
[8.00] Paciente_3 inicia registro
[8.64] Paciente_2 termina consulta (duró 0.64 min)
[8.64] Paciente_2 sale de la clínica (tiempo total: 3.41 min)
[9.10] Paciente_3 termina registro (duró 1.10 min)
[9.10] Paciente_3 solicita médico
[9.10] Paciente_3 inicia consulta médica
[9.31] Paciente_3 termina consulta (duró 0.21 min)
[9.31] Pacie

### Análisis de resultados

In [21]:
if tiempos_de_espera_totales:
    num_pacientes_completados = len(tiempos_de_espera_totales)
    tiempo_promedio = statistics.mean(tiempos_de_espera_totales)
    tiempo_maximo = max(tiempos_de_espera_totales)
    tiempo_minimo = min(tiempos_de_espera_totales)
    desviacion_std = statistics.stdev(tiempos_de_espera_totales) if len(tiempos_de_espera_totales) > 1 else 0
    
    print(f"MÉTRICAS DE RENDIMIENTO:")
    print(f" Total de pacientes completados: {num_pacientes_completados}")
    print(f" Tiempo promedio en clínica: {tiempo_promedio:.2f} minutos")
    print(f" Tiempo máximo en clínica: {tiempo_maximo:.2f} minutos")
    print(f" Tiempo mínimo en clínica: {tiempo_minimo:.2f} minutos")
    print(f" Desviación estándar: {desviacion_std:.2f} minutos")
    
    # Análisis adicional
    print(f"\nANÁLISIS ADICIONAL:")
    print(f" Tasa de llegadas teórica: {60/TASA_LLEGADA_PACIENTES:.2f} pacientes/hora")
    print(f" Tasa real de completados: {(num_pacientes_completados/TIEMPO_SIMULACION)*60:.2f} pacientes/hora")
    
else:
    print("\nNo se completó ningún paciente durante la simulación.")

MÉTRICAS DE RENDIMIENTO:
 Total de pacientes completados: 23
 Tiempo promedio en clínica: 10.82 minutos
 Tiempo máximo en clínica: 29.28 minutos
 Tiempo mínimo en clínica: 2.42 minutos
 Desviación estándar: 6.90 minutos

ANÁLISIS ADICIONAL:
 Tasa de llegadas teórica: 12.00 pacientes/hora
 Tasa real de completados: 11.50 pacientes/hora
